# Predict glucose response

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import os

In [3]:
from datetime import datetime
# headers = [date,carbs,fat]
# dtypes = [datetime, float, float]
date_parser = pd.to_datetime
mat = pd.read_csv('Workshop/REM002/2-voeding.csv', parse_dates=['date'], date_parser = date_parser)


In [4]:
mat

,date,carbs,fat
0,2019-09-06 07:30:00,31.135,13.040
1,2019-09-06 09:30:00,26.595,16.800
2,2019-09-06 11:20:00,0.125,0.000
3,2019-09-06 12:45:00,24.192,0.128
4,2019-09-06 13:30:00,0.595,0.150
...,...,...,...
85,2019-09-19 09:30:00,1.190,0.300
86,2019-09-19 10:30:00,15.795,0.405
87,2019-09-19 12:30:00,88.348,39.228
88,2019-09-19 18:30:00,46.800,22.700


In [5]:
gluc = pd.read_csv('Workshop/REM002/REM002-glucose.txt', sep='\t', parse_dates=['Tijd'], date_parser = date_parser).filter(['Tijd','Historie glucose (mmol/L)'], axis=1)


In [6]:
gluc.rename(columns = {'Tijd': 'date', "Historie glucose (mmol/L)": "glucose"}, inplace = True)

In [7]:
gluc["glucose"] = gluc["glucose"].apply(lambda x: x.replace(",", "."))

In [8]:
gluc

,date,glucose
0,2019-09-06 14:15:00,9.3
1,2019-09-06 14:30:00,8.6
2,2019-09-06 14:45:00,8.6
3,2019-09-06 15:00:00,9.1
4,2019-09-06 15:15:00,9.3
...,...,...
1285,2019-09-20 09:17:00,7.1
1286,2019-09-20 09:32:00,7.1
1287,2019-09-20 09:47:00,7.1
1288,2019-09-20 10:02:00,7.6


In [9]:
df = pd.merge(mat, gluc, how="outer", on="date", sort = True)

In [10]:
df['carbs'].fillna(0, inplace=True)

In [11]:
df['carbs']

0       31.135
1       26.595
2        0.125
3       24.192
4        0.595
         ...  
1366     0.000
1367     0.000
1368     0.000
1369     0.000
1370     0.000
Name: carbs, Length: 1371, dtype: float64

In [12]:
df['fat'].fillna(0, inplace=True)

In [13]:
df['glucose'] = df['glucose'].astype(float).interpolate()

In [193]:
def compute_max_in_interval(xs: pd.Series, start: str, end: str) -> float:
    """Compute the max value for a given interval in a Series"""
    return xs[str(start): str(end)].max()

In [194]:
def add_n_hours(x: pd.Timestamp, n: int) -> pd.Timestamp:
    """Generate a new timestamp adding n hour to the input argument"""
    hour = x.hour + n
    day = x.day
    if hour > 23:
       hour = hour - 23
       day = x.day + 1
    return pd.Timestamp(year = x.year, month = x.month, day = day, hour = hour, minute = x.minute)        

### Create a new time series DataFrame using the date as index

In [195]:
time_series = df.copy()
time_series.set_index("date", inplace=True, drop=True)
time_series.head()

,carbs,fat,glucose
date,,,
2019-09-06 07:30:00,31.135,13.040,NaN
2019-09-06 09:30:00,26.595,16.800,NaN
2019-09-06 11:20:00,0.125,0.000,NaN
2019-09-06 12:45:00,24.192,0.128,NaN
2019-09-06 13:30:00,0.595,0.150,NaN


In [196]:
# find carbs index (when non zero)
carb_index = time_series.carbs[time_series.carbs > 0].index
compute_max_in_interval(time_series["glucose"], carb_index[0], add_n_hours(carb_index[0], 8))


9.4

In [201]:
time_series["max_glucose"] = time_series.apply(lambda x: compute_max_in_interval(time_series["glucose"], x.name, add_n_hours(x.name, 2)) if x.name in carb_index else 0, axis=1)
time_series.head(30)

,carbs,fat,glucose,max_glucose
date,,,,
2019-09-06 07:30:00,31.135,13.040,NaN,NaN
2019-09-06 09:30:00,26.595,16.800,NaN,NaN
2019-09-06 11:20:00,0.125,0.000,NaN,NaN
2019-09-06 12:45:00,24.192,0.128,NaN,9.3
2019-09-06 13:30:00,0.595,0.150,NaN,9.4
2019-09-06 14:15:00,0.000,0.000,9.3,0.0
2019-09-06 14:30:00,0.000,0.000,8.6,0.0
2019-09-06 14:45:00,0.000,0.000,8.6,0.0
2019-09-06 15:00:00,0.000,0.000,9.1,0.0
